In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=2.0, upper=2.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=0.2, upper = 0.2)
    image = tf.image.random_hue(image, max_delta = 0.8)
    image = tf.image.random_saturation(image, lower=0.2, upper=0.2)
    return image, label

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=7,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 16:08:00.372830: {'loss': 1.8948131281137466, 'accuracy': 0.54976875, 'val_loss': 1.8253279411868684, 'val_accuracy': 0.5645}
Epoch 1 ends at 16:11:57.713600: {'loss': 0.933191196680069, 'accuracy': 0.7292625, 'val_loss': 0.7479450645720124, 'val_accuracy': 0.7634}
Epoch 2 ends at 16:16:02.826167: {'loss': 0.46422039376795293, 'accuracy': 0.84065, 'val_loss': 0.7840870318898729, 'val_accuracy': 0.7536}
Epoch 3 ends at 16:20:19.465481: {'loss': 0.2680863493025303, 'accuracy': 0.9059188, 'val_loss': 0.8177522181705305, 'val_accuracy': 0.7786}
Epoch 4 ends at 16:24:41.571255: {'loss': 0.17881050588600336, 'accuracy': 0.9375188, 'val_loss': 0.806446906013094, 'val_accuracy': 0.7948}
Epoch 5 ends at 16:29:05.409609: {'loss': 0.13609543774612248, 'accuracy': 0.953225, 'val_loss': 0.8340318506690347, 'val_accuracy': 0.8132}
Epoch 6 ends at 16:33:28.043135: {'loss': 0.11413427741676568, 'accuracy': 0.9609125, 'val_loss': 0.9698422587221596, 'val_accuracy': 0.8026}


In [12]:
model.evaluate(test_data)

157/157 [==============================]0.9969 - accuracy: 0.812 - 0s 85ms/step - loss: 0.9285 - accuracy: 0.812 - 0s 72ms/step - loss: 0.9333 - accuracy: 0.82 - 0s 65ms/step - loss: 0.9512 - accuracy: 0.82 - 0s 61ms/step - loss: 0.8918 - accuracy: 0.84 - 0s 60ms/step - loss: 0.8361 - accuracy: 0.85 - 0s 57ms/step - loss: 0.8520 - accuracy: 0.84 - 0s 54ms/step - loss: 1.0093 - accuracy: 0.83 - 0s 53ms/step - loss: 0.9607 - accuracy: 0.82 - 1s 52ms/step - loss: 1.0049 - accuracy: 0.82 - 1s 51ms/step - loss: 1.0134 - accuracy: 0.82 - 1s 50ms/step - loss: 0.9840 - accuracy: 0.82 - 1s 50ms/step - loss: 1.0124 - accuracy: 0.82 - 1s 49ms/step - loss: 1.0136 - accuracy: 0.82 - 1s 48ms/step - loss: 1.0252 - accuracy: 0.81 - 1s 48ms/step - loss: 1.0590 - accuracy: 0.81 - 1s 47ms/step - loss: 1.0940 - accuracy: 0.81 - 1s 47ms/step - loss: 1.1237 - accuracy: 0.80 - 1s 47ms/step - loss: 1.0995 - accuracy: 0.80 - 1s 47ms/step - loss: 1.0937 - accuracy: 0.80 - 1s 47ms/step - loss: 1.0581 - accuracy:

[1.0354013911857727, 0.7952]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))